In [7]:
import sqlite3
import pandas as pd
import datetime
import numpy as np
import math
import os

In [8]:
files = os.listdir()
number = []
for i in files:
    if i.endswith('.csv'):
        number.append(i[-7:-4])
        
#number.remove('405')
#number.remove('251')

number
   

['219',
 '220',
 '221',
 '222',
 '224',
 '225',
 '226',
 '227',
 '240',
 '241',
 '242',
 '244',
 '245',
 '246',
 '248',
 '249',
 '251',
 '252',
 '253',
 '254',
 '255',
 '256',
 '277',
 '279',
 '280',
 '283',
 '301',
 '302',
 '303',
 '304',
 '305',
 '307',
 '401',
 '403',
 '404',
 '405']

In [9]:
d = {}
for i in number:
    for file in files:
        if file.endswith('.csv'):
            if i in file:
                csv = file
                
        elif file.endswith('.db'):
            if i in file:
                db = file
    d[csv] = db
                


In [10]:
l = [4,3,2,1]
l.sort()
l

[1, 2, 3, 4]

In [12]:
counter = 0
HOPE = pd.DataFrame()
for pair in d:
    trials = pd.read_csv(pair,index_col = 0)
    trials = trials.loc[trials['block'] > 5]
    trials = trials.reset_index()
    con = sqlite3.connect(d[pair])
    if pair[-7:-4] == '251':
        answers_valence = pd.read_sql_query("SELECT * FROM answers WHERE questionnaire_number != 2 AND question = 0 AND questionnaire_name = 'שאלון מצב-רוח' AND answer LIKE '%Valence%'", con)
    elif pair[-7:-4] == '405':
        answers_valence = pd.read_sql_query("SELECT * FROM answers WHERE questionnaire_number != 83 AND question = 0 AND questionnaire_name = 'שאלון מצב-רוח' AND answer LIKE '%Valence%'", con)
    else:
        answers_valence = pd.read_sql_query("SELECT * FROM answers WHERE question = 0 AND questionnaire_name = 'שאלון מצב-רוח' AND answer LIKE '%Valence%'", con)
    rest = pd.read_sql_query("SELECT * FROM questionnaires WHERE description = '5 דקות מדידה במנוחה'",con)
    mood = pd.read_sql_query("SELECT * FROM questionnaires WHERE description = 'שאלון מצב-רוח'",con)
    stimuli = pd.read_sql_query("SELECT * FROM stimuli", con)
    ans_sad = pd.read_sql_query("SELECT * FROM answers WHERE question = 1 AND questionnaire_name = 'שאלון מצב-רוח' AND answer LIKE '%Sad%'", con)
    anxious = []
    elated = []
    sad = []
    irritable = []
    energetic = []
    for i in range(len(ans_sad['answer'])):
        anxious.append(int(ans_sad['answer'][i].split(',')[0].split('=')[1]))
        elated.append(int(ans_sad['answer'][i].split(',')[1].split('=')[1]))
        sad.append(int(ans_sad['answer'][i].split(',')[2].split('=')[1]))
        irritable.append(int(ans_sad['answer'][i].split(',')[3].split('=')[1]))
        energetic.append(int(ans_sad['answer'][i].split(',')[4].split('=')[1]))

    
    answers_valence['anxious'] = anxious
    answers_valence['elated'] = elated
    answers_valence['sad'] = sad
    answers_valence['irritable'] = irritable
    answers_valence['energetic'] = energetic
    date = []
    for i in answers_valence['answer_time']:
        date.append(pd.to_datetime(datetime.datetime.fromtimestamp(i/1000.0)))
    answers_valence['date'] = date
    answers_valence['date'] = answers_valence['date'].dt.floor('Min')

    date = []
    for i in trials['scheduled_time']:
        date.append(pd.to_datetime(datetime.datetime.fromtimestamp(i/1000.0)))
    trials['sch_date'] = date
    trials['sch_date'] = trials['sch_date'].dt.floor('Min')
    
    valence = []
    arousal = [] 
    for i in range(len(answers_valence['answer'])):
        valence.append(int(answers_valence['answer'][i].split(',')[0].split('=')[1]))
        arousal.append(int(answers_valence['answer'][i].split(',')[1].split('=')[1]))

    answers_valence['valence'] = valence
    answers_valence['arousal'] = arousal 
    
    
    date = []
    for i in rest['scheduled_time']:
        date.append(pd.to_datetime(datetime.datetime.fromtimestamp(i/1000.0)))
    rest['sch_date'] = date
    rest['sch_date'] = rest['sch_date'].dt.floor('Min')

    date = []
    for i in mood['scheduled_time']:
        date.append(pd.to_datetime(datetime.datetime.fromtimestamp(i/1000.0)))
    mood['sch_date'] = date
    mood['sch_date'] = mood['sch_date'].dt.floor('Min')

    date = []
    for i in mood['start_time']:
        if math.isnan(i):
            date.append(i)
        else:
            date.append(pd.to_datetime(datetime.datetime.fromtimestamp(i/1000.0)))
    mood['start_date'] = date
    mood['start_date'] = mood['start_date'].dt.floor('Min')

    date = []
    for i in mood['completed_time']:
        if math.isnan(i):
            date.append(i)
        else:
            date.append(pd.to_datetime(datetime.datetime.fromtimestamp(i/1000.0)))
    mood['com_date'] = date
    mood['com_date'] = mood['com_date'].dt.floor('Min')
    
    
    l = []
    for j in range(len(mood['sch_date'])):
        for i in range(len(rest['sch_date'])):
            if rest['sch_date'][i] == mood['sch_date'][j]:
                l.append(mood['com_date'][j])
                
                
    answers_valence['NoonDate'] = answers_valence['date']
    #for i in range(len(answers_valence['NoonDate'])):
    #    if answers_valence['NoonDate'][i] not in l:
     #       answers_valence['NoonDate'][i] = float("NaN")
    
    for index, row in answers_valence.iterrows():
        if row.NoonDate not in l:
            row.NoonDate = float("NaN")


    df = answers_valence.dropna()
    df = df[1:]
    df = df.reset_index()
    
    
    df['NoonDate'] = df['NoonDate'].dt.floor('d')
    trials['sch_date'] = trials['sch_date'].dt.floor('d')

    trials['valence_memory'] = np.nan
    trials['arousal'] = np.nan
    trials['anxious'] = np.nan
    trials['elated'] = np.nan
    trials['sad'] = np.nan
    trials['irritable'] = np.nan
    trials['energetic'] = np.nan
    for i in range(len(df['NoonDate'])):
        trials.loc[(trials['sch_date'] == df['NoonDate'][i]),['valence_memory']] = int(df['valence'][i])
        trials.loc[(trials['sch_date'] == df['NoonDate'][i]),['arousal']] = int(df['arousal'][i])
        trials.loc[(trials['sch_date'] == df['NoonDate'][i]),['anxious']] = int(df['anxious'][i])
        trials.loc[(trials['sch_date'] == df['NoonDate'][i]),['elated']] = int(df['elated'][i])
        trials.loc[(trials['sch_date'] == df['NoonDate'][i]),['sad']] = int(df['sad'][i])
        trials.loc[(trials['sch_date'] == df['NoonDate'][i]),['irritable']] = int(df['irritable'][i])
        trials.loc[(trials['sch_date'] == df['NoonDate'][i]),['energetic']] = int(df['energetic'][i])

    trials['valence_memory'] = pd.to_numeric(trials['valence_memory'])
    trials['arousal'] = pd.to_numeric(trials['arousal'])
    trials['anxious'] = pd.to_numeric(trials['anxious'])
    trials['elated'] = pd.to_numeric(trials['elated'])
    trials['sad'] = pd.to_numeric(trials['sad'])
    trials['irritable'] = pd.to_numeric(trials['irritable'])
    trials['energetic'] = pd.to_numeric(trials['energetic'])


    
    df1 = trials.drop(columns = ['stim2'])
    df1 = df1.rename(columns={'stim1': 'stim'})
    df2 = trials.drop(columns = ['stim1'])
    df2 = df2.rename(columns={'stim2': 'stim'})
    dfFinal = df1.append(df2)
    dfFinal = dfFinal.loc[dfFinal['stim'] > 17]
    fb = pd.DataFrame(dfFinal.loc[dfFinal['feedback']==1])
    fbs = fb[['block','stim','scheduled_time','sch_date','choice_time','valence_memory','arousal', 'anxious', 'elated', 'sad', 'irritable',
    'energetic']].drop_duplicates()
    fbs['block1'] = np.nan
    fbs['block2'] = np.nan
   
    
    mv = pd.DataFrame(fbs.groupby('stim')['valence_memory'].mean())
    md = pd.DataFrame(fbs.groupby('stim')['choice_time'].max())
    mdv = md.join(mv,on = 'stim')
    mv = pd.DataFrame(fbs.groupby('stim')['arousal'].mean())
    mdv = mdv.join(mv,on = 'stim')
    mv = pd.DataFrame(fbs.groupby('stim')['anxious'].mean())
    mdv = mdv.join(mv,on = 'stim')
    mv = pd.DataFrame(fbs.groupby('stim')['elated'].mean())
    mdv = mdv.join(mv,on = 'stim')
    mv = pd.DataFrame(fbs.groupby('stim')['sad'].mean())
    mdv = mdv.join(mv,on = 'stim')
    mv = pd.DataFrame(fbs.groupby('stim')['irritable'].mean())
    mdv = mdv.join(mv,on = 'stim')
    mv = pd.DataFrame(fbs.groupby('stim')['energetic'].mean())
    mdv = mdv.join(mv,on = 'stim')
    mdv['block1'] = np.nan
    mdv['block2'] = np.nan
    mdv = mdv.reset_index()
    for i in fbs['stim']:
        l = []
        for j in fbs.loc[fbs['stim'] == i]['block']:
            l.append(j)
        s = set(l)
        l = list(s)
        l.sort()
        mdv.loc[mdv['stim'] == i,'block1'] = l[0]
        if len(l) == 2:   
            mdv.loc[mdv['stim'] == i,'block2'] = l[1]


    
    #mdv = mdv.merge(fbs[['block1','block2']], left_on = 'stim',right_on = 'stim',how = 'left')
    dfFinal['stim'] = pd.to_numeric(dfFinal['stim'])
    dfFinal = dfFinal.reset_index()

    trials = trials.merge(mdv,left_on = 'stim1', right_on = 'stim',how = 'left')
    trials = trials.rename(columns = {'valence_memory_y':'valence_stim1','arousal_y':'arousal_stim1',
                                      'anxious_y':'anxious_stim1','elated_y':'elated_stim1',
                                     'sad_y':'sad_stim1','irritable_y':'irritable_stim1','energetic_y':'energetic_stim1',
                                      'choice_time_y':'choice_stim1','choice_time_x':'choice_time'
                                      ,'valence_memory_x':'valence_memory', 'arousal_x':'arousal', 'anxious_x':'anxious',
                                      'elated_x':'elated', 'sad_x':'sad',
                               'irritable_x':'irritable', 'energetic_x':'energetic',
                                      'block1': 'block1_stim1','block2': 'block2_stim1'})

    trials = trials.merge(mdv,left_on = 'stim2', right_on = 'stim',how = 'left')
    trials = trials.rename(columns = {'valence_memory_y':'valence_stim2','arousal_y':'arousal_stim2',
                                      'anxious_y':'anxious_stim2','elated_y':'elated_stim2',
                                     'sad_y':'sad_stim2','irritable_y':'irritable_stim2','energetic_y':'energetic_stim2',
                                      'choice_time_y':'choice_stim2','choice_time_x':'choice_time'
                                      ,'valence_memory_x':'valence_memory', 'arousal_x':'arousal', 'anxious_x':'anxious',
                                      'elated_x':'elated', 'sad_x':'sad',
                               'irritable_x':'irritable', 'energetic_x':'energetic',
                                      'block1': 'block1_stim2','block2': 'block2_stim2'})

    trials['valence_diff_during_learning'] = abs(trials['valence_stim1'] - trials['valence_stim2'])
    trials['valence_mean_1+2'] = (trials['valence_stim1'] + trials['valence_stim2'])/2
    trials['diff_valence'] = abs(trials['valence_mean_1+2'] - trials['valence_memory'])

    trials['arousal_diff_during_learning'] = abs(trials['arousal_stim1'] - trials['arousal_stim2'])
    trials['arousal_mean_1+2'] = (trials['arousal_stim1'] + trials['arousal_stim2'])/2
    trials['diff_arousal'] = abs(trials['arousal_mean_1+2'] - trials['arousal'])

    trials['anxious_diff_during_learning'] = abs(trials['anxious_stim1'] - trials['anxious_stim2'])
    trials['anxious_mean_1+2'] = (trials['anxious_stim1'] + trials['anxious_stim2'])/2
    trials['diff_anxious'] = abs(trials['anxious_mean_1+2'] - trials['anxious'])

    trials['elated_diff_during_learning'] = abs(trials['elated_stim1'] - trials['elated_stim2'])
    trials['elated_mean_1+2'] = (trials['elated_stim1'] + trials['elated_stim2'])/2
    trials['diff_elated'] = abs(trials['elated_mean_1+2'] - trials['elated'])                                            


    trials['sad_diff_during_learning'] = abs(trials['sad_stim1'] - trials['sad_stim2'])
    trials['sad_mean_1+2'] = (trials['sad_stim1'] + trials['sad_stim2'])/2
    trials['diff_sad'] = abs(trials['sad_mean_1+2'] - trials['sad'])

    trials['irritable_diff_during_learning'] = abs(trials['irritable_stim1'] - trials['irritable_stim2'])
    trials['irritable_mean_1+2'] = (trials['irritable_stim1'] + trials['irritable_stim2'])/2
    trials['diff_irritable'] = abs(trials['irritable_mean_1+2'] - trials['irritable'])

    trials['energetic_diff_during_learning'] = abs(trials['energetic_stim1'] - trials['energetic_stim2'])
    trials['energetic_mean_1+2'] = (trials['energetic_stim1'] + trials['energetic_stim2'])/2
    trials['diff_energetic'] = abs(trials['energetic_mean_1+2'] - trials['energetic'])


    trials['last_time_learning_old'] = np.nan
    trials['last_time_learning_new'] = np.nan
    #for i in range(len(trials['last_time_learning_old'])):
     #   trials['last_time_learning_old'][i] = min([trials['choice_stim1'][i],trials['choice_stim2'][i]])
      #  trials['last_time_learning_new'][i] = max([trials['choice_stim1'][i],trials['choice_stim2'][i]])
    for index, row in trials.iterrows():
        row.last_time_learning_old = min([row.choice_stim1,row.choice_stim2])
        row.last_time_learning_new = max([row.choice_stim1,row.choice_stim2])


    trials['time_old'] = trials['choice_time'] - trials['last_time_learning_old']
    trials['time_new'] = trials['choice_time'] - trials['last_time_learning_new']

    FinalTrials = trials.loc[trials['stim1'] > 17].loc[trials['stim2'] > 17].loc[trials['feedback'] == 0][['block','trial','stim1','stim2'
                                                ,'feedback','valence_memory','valence_stim1','valence_stim2','valence_diff_during_learning',
                                                            'valence_mean_1+2','diff_valence','arousal','arousal_stim1', 'arousal_stim2', 'arousal_diff_during_learning',
       'arousal_mean_1+2', 'diff_arousal','anxious', 'anxious_stim1', 'anxious_stim2',
       'anxious_diff_during_learning', 'anxious_mean_1+2', 'diff_anxious','elated','elated_stim1', 'elated_stim2', 'elated_diff_during_learning',
       'elated_mean_1+2', 'diff_elated','sad', 'sad_stim1', 'sad_stim2',
       'sad_diff_during_learning', 'sad_mean_1+2', 'diff_sad','irritable', 'irritable_stim1', 'irritable_stim2', 'irritable_diff_during_learning',
       'irritable_mean_1+2', 'diff_irritable','energetic','energetic_stim1',
       'energetic_stim2', 'energetic_diff_during_learning',
       'energetic_mean_1+2', 'diff_energetic','accuracy','relative_accuracy','time_old','time_new','block1_stim1','block2_stim1'
                                                                            ,'block1_stim2','block2_stim2']]
    FinalTrials = FinalTrials.reset_index()
    FinalTrials = FinalTrials.drop(columns = 'index')
    FinalTrials = FinalTrials.rename(columns = {'accuracy': 'objective accuracy','arousal':'arousal_memory','anxious':'anxious_memory',
                                               'elated':'elated_memory','sad':'sad_memory','irritable':'irritable_memory','energetic':'energetic_memory'})
    FinalTrials['time_in_H_old'] = np.nan
    FinalTrials['time_in_H_new'] = np.nan
    
    #for i in range(len(FinalTrials['time_in_H_old'])):
     #   FinalTrials['time_in_H_old'][i] = FinalTrials['time_old'][i]/3600000
      #  FinalTrials['time_in_H_new'][i] = FinalTrials['time_new'][i]/3600000
        
    for index, row in FinalTrials.iterrows():
        row.time_in_H_old = row.time_old/3600000
        row.time_in_H_new = row.time_new/3600000


    FinalTrials['Participant Number'] = pair[-7:-4]    
    HOPE = HOPE.append(FinalTrials)
    counter +=1
print(counter)

36


In [65]:
#HOPE.to_csv('HopeReal.csv')

In [6]:
HOPE

,block,trial,stim1,stim2,feedback,valence_memory,valence_stim1,valence_stim2,valence_diff_during_learning,valence_mean_1+2,...,relative_accuracy,time_old,time_new,block1_stim1,block2_stim1,block1_stim2,block2_stim2,time_in_H_old,time_in_H_new,Participant Number
0,14,2,23,18,0,-430.0,188.0,151.5,36.5,169.75,...,False,2.838271e+08,1.941558e+08,9.0,10.0,6.0,8.0,78.840849,53.932157,219
1,14,5,18,23,0,-430.0,151.5,188.0,36.5,169.75,...,False,2.838474e+08,1.941761e+08,6.0,8.0,9.0,10.0,78.846508,53.937815,219
2,14,8,20,23,0,-430.0,151.5,188.0,36.5,169.75,...,True,2.838678e+08,1.941965e+08,7.0,8.0,9.0,10.0,78.852171,53.943479,219
3,14,11,23,20,0,-430.0,188.0,151.5,36.5,169.75,...,True,2.838876e+08,1.942163e+08,9.0,10.0,7.0,8.0,78.857673,53.948981,219
4,14,14,18,22,0,-430.0,151.5,188.0,36.5,169.75,...,True,2.839062e+08,1.938016e+08,6.0,8.0,9.0,11.0,78.862819,53.833777,219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,61,67,22,43,0,92.0,158.5,159.0,0.5,158.75,...,NaN,2.164221e+09,1.559964e+09,9.0,11.0,22.0,24.0,601.172405,433.323199,405
1136,61,68,34,43,0,92.0,173.0,159.0,14.0,166.00,...,True,1.818054e+09,1.559969e+09,17.0,19.0,22.0,24.0,505.014914,433.324773,405
1137,61,69,22,53,0,92.0,158.5,-31.0,189.5,63.75,...,False,2.164233e+09,1.299307e+09,9.0,11.0,29.0,30.0,601.175821,360.918598,405
1138,61,70,53,20,0,92.0,-31.0,151.5,182.5,60.25,...,NaN,2.255041e+09,1.299314e+09,29.0,30.0,7.0,8.0,626.400212,360.920491,405


In [11]:
h = list(HOPE.columns)
g = h[5:-11]
HOPE.dropna(subset = g).to_csv('withoutNA.csv')

In [61]:
FinalTrials

,block,trial,stim1,stim2,feedback,valence_memory,valence_stim1,valence_stim2,valence_diff_during_learning,valence_mean_1+2,...,diff_energetic,objective accuracy,relative_accuracy,time,block1_stim1,block2_stim1,block1_stim2,block2_stim2,time_in_H,Participant Number
0,14,2,18,23,0,-2.0,151.5,158.5,7.0,155.00,...,5.00,True,True,2.693779e+08,8.0,6.0,9.0,10.0,74.827183,405
1,14,5,23,18,0,-2.0,158.5,151.5,7.0,155.00,...,5.00,True,True,2.693956e+08,9.0,10.0,8.0,6.0,74.832110,405
2,14,8,23,20,0,-2.0,158.5,151.5,7.0,155.00,...,5.00,NaN,NaN,2.694075e+08,9.0,10.0,8.0,7.0,74.835408,405
3,14,11,18,20,0,-2.0,151.5,151.5,0.0,151.50,...,12.50,True,True,2.694320e+08,8.0,6.0,8.0,7.0,74.842214,405
4,14,14,18,20,0,-2.0,151.5,151.5,0.0,151.50,...,12.50,True,True,2.694486e+08,8.0,6.0,8.0,7.0,74.846841,405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,61,67,22,43,0,92.0,158.5,159.0,0.5,158.75,...,10.50,NaN,NaN,2.164221e+09,9.0,11.0,24.0,22.0,601.172405,405
1136,61,68,34,43,0,92.0,173.0,159.0,14.0,166.00,...,4.25,NaN,True,1.818054e+09,17.0,19.0,24.0,22.0,505.014914,405
1137,61,69,22,53,0,92.0,158.5,-31.0,189.5,63.75,...,3.50,False,False,2.164233e+09,9.0,11.0,29.0,30.0,601.175821,405
1138,61,70,53,20,0,92.0,-31.0,151.5,182.5,60.25,...,11.00,NaN,NaN,2.255041e+09,29.0,30.0,8.0,7.0,626.400212,405


In [64]:
print(len(set([i for i in HOPE['Participant Number']])))

36


In [68]:
HOPE.describe()

,block,trial,stim1,stim2,feedback,valence_memory,valence_stim1,valence_stim2,valence_diff_during_learning,valence_mean_1+2,...,irritable_mean_1+2,diff_irritable,energetic_memory,energetic_stim1,energetic_stim2,energetic_diff_during_learning,energetic_mean_1+2,diff_energetic,time,time_in_H
count,40910.000000,40910.000000,40910.000000,40910.000000,40910.0,39269.000000,40830.000000,40850.000000,40772.000000,40772.000000,...,40772.000000,39167.000000,39269.000000,40830.000000,40850.000000,40772.000000,40772.000000,39167.000000,4.091000e+04,40910.000000
mean,41.300538,35.037326,50.230579,50.221902,0.0,197.279865,191.477791,192.652793,149.663967,192.279601,...,6.786273,6.172382,34.174794,34.874831,35.200373,14.536878,35.035000,15.404992,7.432428e+08,206.456330
std,13.729414,20.489489,20.535371,20.540716,0.0,205.529853,164.992477,165.301697,144.257607,127.976189,...,7.628194,7.706465,23.276279,20.982679,21.035661,12.918622,18.625882,12.833109,4.549202e+08,126.366729
min,14.000000,0.000000,18.000000,18.000000,0.0,-494.000000,-478.000000,-478.000000,0.000000,-327.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.962603e+07,16.562786
25%,30.000000,17.000000,33.000000,33.000000,0.0,99.000000,98.500000,99.500000,39.500000,110.750000,...,0.750000,1.000000,16.000000,19.000000,19.000000,4.500000,21.250000,5.500000,4.319150e+08,119.976392
50%,42.000000,35.000000,49.000000,49.000000,0.0,249.000000,213.000000,213.500000,108.000000,201.750000,...,4.500000,3.500000,29.000000,31.500000,32.000000,11.000000,32.000000,12.000000,6.847338e+08,190.203835
75%,54.000000,53.000000,66.000000,66.000000,0.0,339.000000,312.000000,314.000000,219.500000,277.500000,...,9.750000,8.750000,48.000000,47.000000,47.500000,21.000000,46.250000,21.750000,8.705370e+08,241.815825
max,61.000000,71.000000,98.000000,98.000000,0.0,499.000000,498.500000,498.500000,934.000000,494.750000,...,47.250000,71.000000,100.000000,100.000000,100.000000,82.000000,100.000000,83.250000,2.604454e+09,723.459555
